In [1]:
# create a simple POW interface so we can test a ton of different proof methods
class POW(object):
    def __init__(self, generator, prover):
        assert generator != None and prover != None, 'Provide a secret generation function and a conditional \
        to test if a proof is valid'
        self.generator = generator
        self.prover = prover
        
    def find(self, payload: bytes):
        while True:
            # generate a new secret to test
            secret = self.generator()

            # test said secret
            if self.prover(payload, secret):
                break

        # return when satisfied
        return secret

    def check(self, payload: bytes, secret: bytes):
        return self.prover(payload, secret)

In [2]:
# a cleaner interface

def gen_nonce():
    return secrets.token_bytes()

def dyno_proof_satisfied(payload: bytes, secret: bytes, difficulty: int=13):
    # calculate the hash of combined payload and secret
    h = hashlib.sha256()
    h.update(payload + secret)
    
    # turn this into a binary substring from index 1 -> the level of difficulty
    BINARY_LITERAL_CLIP = 3
    x = byte_to_int(h.digest())
    x = bin(x)
    x = x[BINARY_LITERAL_CLIP:BINARY_LITERAL_CLIP+difficulty]
    
    # create a string full of zeros the length of the level of difficulty
    y = ''.join(['0' for x in range(difficulty)])
    
    # return the comparison
    return x == y

def find_proof(payload: bytes, generator=gen_nonce, condition=None):
    while True:
        # generate a new secret to test
        secret = generator()
        
        # test said secret
        if condition(payload, secret):
            break
        
    # return when satisfied
    return secret

In [3]:
def string_satisfies_mud(s: str, difficulty: int=10, mud: int=2):
    assert len(s) == difficulty, 'Pass in a proof string the length of the true difficulty'
    l = [int(ss) for ss in s]
    return sum(l) <= mud